In [1]:
import numpy as np
import pandas as pd
import doubleml as dml
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import mean_squared_error, log_loss
from sklearn.model_selection import train_test_split, KFold
from doubleml._utils_resampling import DoubleMLResampling

## PLR Model

In [3]:
## with ACIC data
n_folds = 4
no_iter = 100

for sc in range(16):
    sc+=1
    res_path = f"../results/acic/plr/XGBoost/Scenario{sc}_"
    res_fullsample = []
    res_splitsample = []
    res_onfolds = []
    for k in range(no_iter):
        #print(k)
        # for saving each iteration
        _res_full = []
        _res_split = []
        _res_of = []
        
        # load data
        df = pd.read_csv(f"..dgp/acic/Scenario{sc}/CHDScenario{sc}DS{k+1}.csv")           
        y, d, X = df["Y"].values, df["A"].values, df.drop(columns=["Y","A"]).values
    
        # full sample, tuning
        n_trees_ml_m, n_trees_ml_l = 0, 0  
        kf = KFold(n_splits=n_folds)
        for _, (train_index, test_index) in enumerate(kf.split(X)):
            tune_l = XGBRegressor(n_estimators=100, max_depth=2, learning_rate=0.1, early_stopping_rounds=8, eval_metric="rmse")
            tune_l.fit(X[train_index], y[train_index], eval_set=[(X[test_index], y[test_index])], verbose=False)
            tune_m = XGBClassifier(n_estimators=100, max_depth=2, learning_rate=0.1, early_stopping_rounds=8, eval_metric="logloss")
            tune_m.fit(X[train_index], d[train_index], eval_set=[(X[test_index], d[test_index])], verbose=False)
            n_trees_ml_m += tune_m.best_ntree_limit
            n_trees_ml_l += tune_l.best_ntree_limit

        ml_l = XGBRegressor(n_estimators= int(n_trees_ml_l / n_folds), max_depth=2, learning_rate=0.1)
        ml_m = XGBClassifier(n_estimators= int(n_trees_ml_m / n_folds), max_depth=2, learning_rate=0.1)

        # full sample, doubleml
        np.random.seed(k)
        obj_dml_data = dml.DoubleMLData(df,y_col='Y',d_cols='A')
        dml_plr = dml.DoubleMLPLR(obj_dml_data, ml_l = ml_l, ml_m = ml_m, n_folds = n_folds)
        dml_plr.fit(store_predictions=True)

        _res_full.append(dml_plr.summary["coef"].values[0])
        _res_full.append(dml_plr.summary["2.5 %"].values[0])
        _res_full.append(dml_plr.summary["97.5 %"].values[0])
        _res_full.append(np.sqrt(tune_l.best_score))
        _res_full.append(tune_m.best_score)
        _res_full.append(mean_squared_error(y, dml_plr.summary["coef"].values[0] * d + dml_plr.predictions["ml_l"][:,0,0]))
        _res_full.append(mean_squared_error(y, dml_plr.predictions["ml_l"][:,0,0]))
        _res_full.append(log_loss(d, dml_plr.predictions["ml_m"][:,0,0]))

        # split sample, tuning
        df_tune, df_test = train_test_split(df, test_size= 0.5, random_state = 42*k)
        y_tune, d_tune, X_tune = df_tune["Y"].values, df_tune["A"].values, df_tune.drop(columns=["Y","A"]).values
        y_test, d_test, X_test = df_test["Y"].values, df_test["A"].values, df_test.drop(columns=["Y","A"]).values
        
        tune_l = XGBRegressor(n_estimators=100, max_depth=2, learning_rate=0.1, early_stopping_rounds=8, eval_metric="rmse")
        tune_l.fit(X_tune, y_tune, eval_set=[(X_test, y_test)], verbose=False)
        tune_m = XGBClassifier(n_estimators=100, max_depth=2, learning_rate=0.1, early_stopping_rounds=8, eval_metric="logloss")
        tune_m.fit(X_tune, d_tune, eval_set=[(X_test, d_test)], verbose=False)

        ml_l = XGBRegressor(n_estimators= tune_l.best_ntree_limit, max_depth=2, learning_rate=0.1)
        ml_m = XGBClassifier(n_estimators= tune_m.best_ntree_limit, max_depth=2, learning_rate=0.1)

        # split sample, doubleml           
        np.random.seed(2*k)
        obj_dml_data = dml.DoubleMLData(df_test, y_col='Y', d_cols='A')
        dml_plr_split = dml.DoubleMLPLR(obj_dml_data, ml_l = ml_l, ml_m = ml_m, n_folds = n_folds)
        dml_plr_split.fit(store_predictions = True)

        _res_split.append(dml_plr_split.summary["coef"].values[0])
        _res_split.append(dml_plr_split.summary["2.5 %"].values[0])
        _res_split.append(dml_plr_split.summary["97.5 %"].values[0])
        _res_split.append(np.sqrt(tune_l.best_score))
        _res_split.append(tune_m.best_score)
        _res_split.append(mean_squared_error(y_test, dml_plr_split.summary["coef"].values[0] * d_test + dml_plr_split.predictions["ml_l"][:,0,0]))
        _res_split.append(mean_squared_error(df_test["Y"], dml_plr_split.predictions["ml_l"][:,0,0]))
        _res_split.append(log_loss(df_test["A"], dml_plr_split.predictions["ml_m"][:,0,0]))

        # on folds
        smpls = DoubleMLResampling(n_folds=n_folds, n_rep=1, n_obs=y.shape[0], apply_cross_fitting=True).split_samples()
        param_list_ml_l = []
        param_list_ml_m = []
        for _, (train_index, test_index) in enumerate(smpls[0]):
            tune_l = XGBRegressor(n_estimators=100, max_depth=2, learning_rate=0.1, early_stopping_rounds=8, eval_metric="rmse")
            tune_l.fit(X[train_index], y[train_index], eval_set=[(X[test_index], y[test_index])], verbose=False)
            tune_m = XGBClassifier(n_estimators=100, max_depth=2, learning_rate=0.1, early_stopping_rounds=8, eval_metric="logloss")
            tune_m.fit(X[train_index], d[train_index], eval_set=[(X[test_index], d[test_index])], verbose=False)
            param_list_ml_l.append({'n_estimators': tune_l.best_ntree_limit, 'max_depth': 2, 'learning_rate': 0.1})
            param_list_ml_m.append({'n_estimators': tune_m.best_ntree_limit, 'max_depth': 2, 'learning_rate': 0.1})
        param_list_ml_l = [param_list_ml_l]
        param_list_ml_m = [param_list_ml_m]

        obj_dml_data = dml.DoubleMLData(df,y_col='Y',d_cols='A')
        ml_l = XGBRegressor()
        ml_m = XGBClassifier()

        np.random.seed(3*k)
        dml_plr_onfolds = dml.DoubleMLPLR(obj_dml_data, ml_l = ml_l, ml_m = ml_m, n_folds = n_folds, draw_sample_splitting=False)
        dml_plr_onfolds.set_sample_splitting(smpls)
        dml_plr_onfolds.set_ml_nuisance_params("ml_l", "A", param_list_ml_l)
        dml_plr_onfolds.set_ml_nuisance_params("ml_m", "A", param_list_ml_m)
        dml_plr_onfolds.fit(store_predictions=True, store_models = True)

        _res_of.append(dml_plr_onfolds.summary["coef"].values[0])
        _res_of.append(dml_plr_onfolds.summary["2.5 %"].values[0])
        _res_of.append(dml_plr_onfolds.summary["97.5 %"].values[0])
        _res_of.append(np.nan)
        _res_of.append(np.nan)
        _res_of.append(mean_squared_error(y, dml_plr_onfolds.summary["coef"].values[0] * d + dml_plr_onfolds.predictions["ml_l"][:,0,0]))
        _res_of.append(mean_squared_error(y, dml_plr_onfolds.predictions["ml_l"][:,0,0]))
        _res_of.append(log_loss(d, dml_plr_onfolds.predictions["ml_m"][:,0,0]))
        
        # # add this iteration to overall results
        res_fullsample.append(_res_full)
        res_splitsample.append(_res_split)
        res_onfolds.append(_res_of)

        # save current result
        pd.DataFrame(res_fullsample, columns = ["coef","2.5%","97.5%","tune_loss_mll","tune_loss_mlm","loss_Y","fs_loss_mll","fs_loss_mlm"]).to_csv(res_path + f"fullsample.csv")
        pd.DataFrame(res_splitsample, columns=["coef","2.5%","97.5%","tune_loss_mll","tune_loss_mlm","loss_Y","fs_loss_mll","fs_loss_mlm"]).to_csv(res_path + f"splitsample.csv")
        pd.DataFrame(res_onfolds, columns=["coef","2.5%","97.5%","tune_loss_mll","tune_loss_mlm","loss_Y","fs_loss_mll","fs_loss_mlm"]).to_csv(res_path + f"onfolds.csv")

## IRM Model

In [ ]:
## with ACIC data
n_folds = 4
no_iter = 100

for sc in range(16):
    sc+=1
    res_path = f"../results/acic/irm/XGBoost/Scenario{sc}_"
    res_fullsample = []
    res_splitsample = []
    res_onfolds = []
    for k in range(no_iter):
        #print(k)
        # for saving each iteration
        _res_full = []
        _res_split = []
        _res_of = []
        
        # load data
        df = pd.read_csv(f"../dgp/acic/Scenario{sc}/CHDScenario{sc}DS{k+1}.csv")           
        y, d, X = df["Y"].values, df["A"].values, df.drop(columns=["Y","A"]).values
    
        # full sample, tuning
        n_trees_ml_m, n_trees_ml_g = 0, 0  
        kf = KFold(n_splits=n_folds)
        for _, (train_index, test_index) in enumerate(kf.split(X)):
            tune_g = XGBRegressor(n_estimators=100, max_depth=2, learning_rate=0.1, early_stopping_rounds=8, eval_metric="rmse")
            tune_g.fit(np.c_[X,d][train_index], y[train_index], eval_set=[(np.c_[X,d][test_index], y[test_index])], verbose=False)
            tune_m = XGBClassifier(n_estimators=100, max_depth=2, learning_rate=0.1, early_stopping_rounds=8, eval_metric="logloss")
            tune_m.fit(X[train_index], d[train_index], eval_set=[(X[test_index], d[test_index])], verbose=False)
            n_trees_ml_m += tune_m.best_ntree_limit
            n_trees_ml_g += tune_g.best_ntree_limit

        ml_g = XGBRegressor(n_estimators= int(n_trees_ml_g / n_folds), max_depth=2, learning_rate=0.1)
        ml_m = XGBClassifier(n_estimators= int(n_trees_ml_m / n_folds), max_depth=2, learning_rate=0.1)

        # full sample, doubleml
        np.random.seed(k)
        obj_dml_data = dml.DoubleMLData(df,y_col='Y',d_cols='A')
        dml_irm = dml.DoubleMLIRM(obj_dml_data, ml_g = ml_g, ml_m = ml_m, n_folds = n_folds)
        dml_irm.fit(store_predictions=True)

        _res_full.append(dml_irm.summary["coef"].values[0])
        _res_full.append(dml_irm.summary["2.5 %"].values[0])
        _res_full.append(dml_irm.summary["97.5 %"].values[0])
        _res_full.append(np.sqrt(tune_g.best_score))
        _res_full.append(tune_m.best_score)
        treat_ind = (df["A"] == 1)
        ml_g_pred = treat_ind * dml_irm.predictions["ml_g1"][:,0,0] + (1 - treat_ind) * dml_irm.predictions["ml_g0"][:,0,0]
        _res_full.append(mean_squared_error(y, ml_g_pred))
        _res_full.append(log_loss(d, dml_irm.predictions["ml_m"][:,0,0]))

        # split sample, tuning
        df_tune, df_test = train_test_split(df, test_size= 0.5, random_state = 42*k)
        y_tune, d_tune, X_tune = df_tune["Y"].values, df_tune["A"].values, df_tune.drop(columns=["Y","A"]).values
        y_test, d_test, X_test = df_test["Y"].values, df_test["A"].values, df_test.drop(columns=["Y","A"]).values
        
        tune_g = XGBRegressor(n_estimators=100, max_depth=2, learning_rate=0.1, early_stopping_rounds=8, eval_metric="rmse")
        tune_g.fit(np.c_[X_tune, d_tune], y_tune, eval_set=[(np.c_[X_test, d_test], y_test)], verbose=False)
        tune_m = XGBClassifier(n_estimators=100, max_depth=2, learning_rate=0.1, early_stopping_rounds=8, eval_metric="logloss")
        tune_m.fit(X_tune, d_tune, eval_set=[(X_test, d_test)], verbose=False)

        ml_g = XGBRegressor(n_estimators= tune_g.best_ntree_limit, max_depth=2, learning_rate=0.1)
        ml_m = XGBClassifier(n_estimators= tune_m.best_ntree_limit, max_depth=2, learning_rate=0.1)

        # split sample, doubleml           
        np.random.seed(2*k)
        obj_dml_data = dml.DoubleMLData(df_test, y_col='Y', d_cols='A')
        dml_irm_split = dml.DoubleMLIRM(obj_dml_data, ml_g = ml_g, ml_m = ml_m, n_folds = n_folds)
        dml_irm_split.fit(store_predictions = True)

        _res_split.append(dml_irm_split.summary["coef"].values[0])
        _res_split.append(dml_irm_split.summary["2.5 %"].values[0])
        _res_split.append(dml_irm_split.summary["97.5 %"].values[0])
        _res_split.append(np.sqrt(tune_g.best_score))
        _res_split.append(tune_m.best_score)
        treat_ind = (df_test["A"] == 1)
        ml_g_pred = treat_ind * dml_irm_split.predictions["ml_g1"][:,0,0] + (1 - treat_ind) * dml_irm_split.predictions["ml_g0"][:,0,0]
        _res_split.append(mean_squared_error(df_test["Y"], ml_g_pred))
        _res_split.append(log_loss(df_test["A"], dml_irm_split.predictions["ml_m"][:,0,0]))

        # on folds
        smpls = DoubleMLResampling(n_folds=n_folds, n_rep=1, n_obs=y.shape[0], apply_cross_fitting=True).split_samples()
        param_list_ml_g = []
        param_list_ml_m = []
        for _, (train_index, test_index) in enumerate(smpls[0]):
            tune_g = XGBRegressor(n_estimators=100, max_depth=2, learning_rate=0.1, early_stopping_rounds=8, eval_metric="rmse")
            tune_g.fit(np.c_[X,d][train_index], y[train_index], eval_set=[(np.c_[X,d][test_index], y[test_index])], verbose=False)
            tune_m = XGBClassifier(n_estimators=100, max_depth=2, learning_rate=0.1, early_stopping_rounds=8, eval_metric="logloss")
            tune_m.fit(X[train_index], d[train_index], eval_set=[(X[test_index], d[test_index])], verbose=False)
            param_list_ml_g.append({'n_estimators': tune_g.best_ntree_limit, 'max_depth': 2, 'learning_rate': 0.1})
            param_list_ml_m.append({'n_estimators': tune_m.best_ntree_limit, 'max_depth': 2, 'learning_rate': 0.1})
        param_list_ml_g = [param_list_ml_g]
        param_list_ml_m = [param_list_ml_m]

        obj_dml_data = dml.DoubleMLData(df,y_col='Y',d_cols='A')

        ml_g = XGBRegressor()
        ml_m = XGBClassifier()

        np.random.seed(3*k)
        dml_irm_onfolds = dml.DoubleMLIRM(obj_dml_data, ml_g = ml_g, ml_m = ml_m, n_folds = n_folds, draw_sample_splitting=False)
        dml_irm_onfolds.set_sample_splitting(smpls)
        dml_irm_onfolds.set_ml_nuisance_params("ml_g0", "A", param_list_ml_g)
        dml_irm_onfolds.set_ml_nuisance_params("ml_g1", "A", param_list_ml_g)
        dml_irm_onfolds.set_ml_nuisance_params("ml_m", "A", param_list_ml_m)
        dml_irm_onfolds.fit(store_predictions=True, store_models = True)

        _res_of.append(dml_irm_onfolds.summary["coef"].values[0])
        _res_of.append(dml_irm_onfolds.summary["2.5 %"].values[0])
        _res_of.append(dml_irm_onfolds.summary["97.5 %"].values[0])
        _res_of.append(np.nan)
        _res_of.append(np.nan)
        treat_ind = (df["A"] == 1)
        ml_g_pred = treat_ind * dml_irm_onfolds.predictions["ml_g1"][:,0,0] + (1 - treat_ind) * dml_irm_onfolds.predictions["ml_g0"][:,0,0]
        _res_of.append(mean_squared_error(y, ml_g_pred))
        _res_of.append(log_loss(d, dml_irm_onfolds.predictions["ml_m"][:,0,0]))
        
        # # add this iteration to overall results
        res_fullsample.append(_res_full)
        res_splitsample.append(_res_split)
        res_onfolds.append(_res_of)

        # save current result
        pd.DataFrame(res_fullsample, columns = ["coef","2.5%","97.5%","tune_loss_mlg","tune_loss_mlm","fs_loss_mlg","fs_loss_mlm"]).to_csv(res_path + f"fullsample.csv")
        pd.DataFrame(res_splitsample, columns=["coef","2.5%","97.5%","tune_loss_mlg","tune_loss_mlm","fs_loss_mlg","fs_loss_mlm"]).to_csv(res_path + f"splitsample.csv")
        pd.DataFrame(res_onfolds, columns=["coef","2.5%","97.5%","tune_loss_mlg","tune_loss_mlm","fs_loss_mlg","fs_loss_mlm"]).to_csv(res_path + f"onfolds.csv")